# PREPARING DATA

Don't use if not necessary

In [54]:
import pandas as pd
import re
import string
#df = pd.read_csv("Tweet_output_fedy-nostopword.csv")
df = pd.read_excel("positif_train_xl.xlsx")
df

,Tweet
0,alhamdulilah orang jalan pt ibrahim daftar vak...
1,antri ambulance bawa pasien covid dijogja mlm ...
2,april banser tanggap bencana bagana bagana kab...
3,as dukung indonesia lawan tingkat covid sindon...
4,as kirim juta vaksin covid moderna indonesia j...
...,...
57,ppkm darurat laku juli laksana tekan sebar cov...
58,ppkm darurat laku patuh tekan laju tular covid...
59,ppkm darurat sesuai instruksi presiden jokowi ...
60,ppkm langkah sejarah ivermectin obat cacing du...


In [55]:
#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean
df

,Tweet
0,alhamdulilah orang jalan pt ibrahim daftar vak...
1,antri ambulance bawa pasien covid dijogja mlm ...
2,april banser tanggap bencana bagana bagana kab...
3,as dukung indonesia lawan tingkat covid sindon...
4,as kirim juta vaksin covid moderna indonesia j...
...,...
57,ppkm darurat laku juli laksana tekan sebar cov...
58,ppkm darurat laku patuh tekan laju tular covid...
59,ppkm darurat sesuai instruksi presiden jokowi ...
60,ppkm langkah sejarah ivermectin obat cacing du...


In [57]:
df['tweet_tokennew'] = df['Tweet'].apply(lambda x: clean_tweets(x))

In [ ]:
df

In [ ]:
#ambil data Tweet(bersih) - make code as a comment if don't use
df.drop(df.columns[[0]], axis = 1, inplace = True)

In [ ]:
df

In [ ]:
df.to_csv('Token_train1.csv',encoding='utf8', index=False)

# TF IDF COUNT

In [ ]:
import pandas as pd 
import numpy as np

In [58]:
TWEET_DATA = pd.read_csv('Token_train1.csv')
#TWEET_DATA = df
TWEET_DATA

,tweet_tokennew
0,"['alhamdulilah', 'orang', 'jalan', 'pt', 'ibra..."
1,"['antri', 'ambulance', 'bawa', 'pasien', 'covi..."
2,"['april', 'banser', 'tanggap', 'bencana', 'bag..."
3,"['as', 'dukung', 'indonesia', 'lawan', 'tingka..."
4,"['as', 'kirim', 'juta', 'vaksin', 'covid', 'mo..."
...,...
57,"['ppkm', 'darurat', 'laku', 'juli', 'laksana',..."
58,"['ppkm', 'darurat', 'laku', 'patuh', 'tekan', ..."
59,"['ppkm', 'darurat', 'sesuai', 'instruksi', 'pr..."
60,"['ppkm', 'langkah', 'sejarah', 'ivermectin', '..."


In [80]:
# convert list formated string to list
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

TWEET_DATA["tweet_list"] = TWEET_DATA["tweet_tokennew"].apply(convert_text_list)


print(TWEET_DATA["tweet_list"][50])

print("\ntype : ", type(TWEET_DATA["tweet_list"][50]))

['pentadbiran', 'amerika', 'syarikat', 'sahkan', 'sumbang', 'juta', 'dos', 'vaksin', 'covid', 'pfizer', 'biontech', 'kpd', 'malaysia', 'pegawai', 'rumah', 'putih', 'sumbang', 'juta', 'vaksin', 'malaysia', 'isnin', 'julai', 'presiden', 'jo', 'biden', 'komited', 'sumbang', 'juta', 'dos', 'vaksin', 'dunia']

type :  <class 'list'>


TF

In [81]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

TWEET_DATA["TF_dict"] = TWEET_DATA['tweet_list'].apply(calc_TF)

TWEET_DATA["TF_dict"].head(10)
print(TWEET_DATA)

                                       tweet_tokennew  \
0   ['alhamdulilah', 'orang', 'jalan', 'pt', 'ibra...   
1   ['antri', 'ambulance', 'bawa', 'pasien', 'covi...   
2   ['april', 'banser', 'tanggap', 'bencana', 'bag...   
3   ['as', 'dukung', 'indonesia', 'lawan', 'tingka...   
4   ['as', 'kirim', 'juta', 'vaksin', 'covid', 'mo...   
..                                                ...   
57  ['ppkm', 'darurat', 'laku', 'juli', 'laksana',...   
58  ['ppkm', 'darurat', 'laku', 'patuh', 'tekan', ...   
59  ['ppkm', 'darurat', 'sesuai', 'instruksi', 'pr...   
60  ['ppkm', 'langkah', 'sejarah', 'ivermectin', '...   
61  ['proning', 'the', 'miracle', 'of', 'sujud', '...   

                                           tweet_list  \
0   [alhamdulilah, orang, jalan, pt, ibrahim, daft...   
1   [antri, ambulance, bawa, pasien, covid, dijogj...   
2   [april, banser, tanggap, bencana, bagana, baga...   
3   [as, dukung, indonesia, lawan, tingkat, covid,...   
4   [as, kirim, juta, vaksin, 

In [82]:
# Check TF result
index = 50

print('%20s' % "term", "\t", "TF\n")
for key in TWEET_DATA["TF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key])

                term 	 TF

         pentadbiran 	 0.03225806451612903
             amerika 	 0.03225806451612903
            syarikat 	 0.03225806451612903
              sahkan 	 0.03225806451612903
             sumbang 	 0.0967741935483871
                juta 	 0.0967741935483871
                 dos 	 0.06451612903225806
              vaksin 	 0.0967741935483871
               covid 	 0.03225806451612903
              pfizer 	 0.03225806451612903
            biontech 	 0.03225806451612903
                 kpd 	 0.03225806451612903
            malaysia 	 0.06451612903225806
             pegawai 	 0.03225806451612903
               rumah 	 0.03225806451612903
               putih 	 0.03225806451612903
               isnin 	 0.03225806451612903
               julai 	 0.03225806451612903
            presiden 	 0.03225806451612903
                  jo 	 0.03225806451612903
               biden 	 0.03225806451612903
             komited 	 0.03225806451612903
               dunia 	 0.03225

IDF

In [83]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(TWEET_DATA["TF_dict"])

In [63]:
DF

{'alhamdulilah': 1,
 'orang': 6,
 'jalan': 2,
 'pt': 2,
 'ibrahim': 1,
 'daftar': 2,
 'vaksin': 5,
 'program': 2,
 'tubuh': 2,
 'team': 1,
 'data': 1,
 'sumbang': 2,
 'rm': 1,
 'sakit': 4,
 'covid': 61,
 'penghulu': 1,
 'mukim': 1,
 'pontin': 1,
 'jpkk': 1,
 'antri': 1,
 'ambulance': 1,
 'bawa': 3,
 'pasien': 1,
 'dijogja': 1,
 'mlm': 1,
 'moga': 9,
 'layan': 2,
 'pulih': 2,
 'semangat': 3,
 'kru': 1,
 'juang': 3,
 'hantar': 1,
 'nakes': 2,
 'rs': 2,
 'jogjakarta': 1,
 'april': 1,
 'banser': 1,
 'tanggap': 2,
 'bencana': 1,
 'bagana': 1,
 'kabupaten': 1,
 'kalong': 1,
 'makam': 1,
 'jenazah': 1,
 'takut': 1,
 'panggil': 2,
 'jiwa': 1,
 'cinta': 1,
 'negeri': 2,
 'as': 2,
 'dukung': 5,
 'indonesia': 8,
 'lawan': 6,
 'tingkat': 4,
 'sindonews': 1,
 'bukanberitabiasa': 1,
 'kirim': 1,
 'juta': 3,
 'moderna': 1,
 'jabat': 2,
 'bahas': 1,
 'rencana': 1,
 'bantu': 8,
 'upaya': 2,
 'respons': 1,
 'luas': 2,
 'atas': 1,
 'lonjak': 1,
 'gubernur': 3,
 'jawa': 3,
 'timur': 2,
 'khofifah': 1,
 'i

# MENGHITUNG IDF

In [84]:
n_document = len(TWEET_DATA)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [71]:
IDF

{'alhamdulilah': 3.4339872044851463,
 'orang': 2.181224235989778,
 'jalan': 3.028522096376982,
 'pt': 3.028522096376982,
 'ibrahim': 3.4339872044851463,
 'daftar': 3.028522096376982,
 'vaksin': 2.3353749158170367,
 'program': 3.028522096376982,
 'tubuh': 3.028522096376982,
 'team': 3.4339872044851463,
 'data': 3.4339872044851463,
 'sumbang': 3.028522096376982,
 'rm': 3.4339872044851463,
 'sakit': 2.517696472610991,
 'covid': 0.0,
 'penghulu': 3.4339872044851463,
 'mukim': 3.4339872044851463,
 'pontin': 3.4339872044851463,
 'jpkk': 3.4339872044851463,
 'antri': 3.4339872044851463,
 'ambulance': 3.4339872044851463,
 'bawa': 2.740840023925201,
 'pasien': 3.4339872044851463,
 'dijogja': 3.4339872044851463,
 'mlm': 3.4339872044851463,
 'moga': 1.824549292051046,
 'layan': 3.028522096376982,
 'pulih': 3.028522096376982,
 'semangat': 2.740840023925201,
 'kru': 3.4339872044851463,
 'juang': 2.740840023925201,
 'hantar': 3.4339872044851463,
 'nakes': 3.028522096376982,
 'rs': 3.028522096376982,

In [85]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
TWEET_DATA["TF-IDF_dict"] = TWEET_DATA["TF_dict"].apply(calc_TF_IDF)

In [86]:
# Check TF-IDF result
index = 50

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in TWEET_DATA["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key] ,"\t" , TWEET_DATA["TF-IDF_dict"][index][key])

                term 	         TF 	              TF-IDF

         pentadbiran 	 0.03225806451612903 	 0.11077378078984343
             amerika 	 0.03225806451612903 	 0.11077378078984343
            syarikat 	 0.03225806451612903 	 0.11077378078984343
              sahkan 	 0.03225806451612903 	 0.11077378078984343
             sumbang 	 0.0967741935483871 	 0.2930827835203531
                juta 	 0.0967741935483871 	 0.2652425829605033
                 dos 	 0.06451612903225806 	 0.22154756157968686
              vaksin 	 0.0967741935483871 	 0.22600402411132614
               covid 	 0.03225806451612903 	 0.0
              pfizer 	 0.03225806451612903 	 0.11077378078984343
            biontech 	 0.03225806451612903 	 0.11077378078984343
                 kpd 	 0.03225806451612903 	 0.09769426117345104
            malaysia 	 0.06451612903225806 	 0.22154756157968686
             pegawai 	 0.03225806451612903 	 0.11077378078984343
               rumah 	 0.03225806451612903 	 0.0884141

In [87]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

TWEET_DATA["TF_IDF_Vec"] = TWEET_DATA["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(TWEET_DATA["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(TWEET_DATA["TF_IDF_Vec"][0]))

print first row matrix TF_IDF_Vec Series

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09483583634738166, 0.0, 0.0, 0.0, 0.20307607963626406, 0.0, 0.0, 0.0, 0.0, 0.10946506402656483, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

matrix size :  50


In [88]:
# Convert Series to List
TF_IDF_Vec_List = np.array(TWEET_DATA["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
5,moga,1.808781
3,ppkm,1.798004
1,sebar,1.649591
4,darurat,1.536947
6,indonesia,1.448438
2,sehat,1.374242
11,lawan,1.214122
8,pandemi,1.158750
14,vaksin,1.132052
33,pagi,1.044130
